In [2]:
# import libraries
import sys
import os
import yaml
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import h5py
import datetime as dt
from jupyterlab_h5web import H5Web
# import modules with the functionalities offered by CompositionSpace
from compositionspace.get_gitrepo_commit import get_repo_last_commit
from compositionspace.preparation import ProcessPreparation
from compositionspace.segmentation import ProcessSegmentation
from compositionspace.clustering import ProcessClustering

In [9]:
def ceil_to_multiple(number, multiple):
    return multiple * np.ceil(number / multiple)

def floor_to_multiple(number, multiple):
    return multiple * np.floor(number / multiple)

# ceil to a multiple of 1.5
# print(ceil_to_multiple(23.0000000000000000000000000000000000000, 1.5))
# floor to a multiple of 1.5
# print(floor_to_multiple(-23.0000000000000000000000000000000000000, 1.5))

In [3]:
# ! pip list  
MY_PROCESSED_DATA_PATH = f"{os.getcwd()}"
print(f"Executing compositionspace in the following working directory: {os.getcwd()}")

Executing compositionspace in the following working directory: /home/kaiobach/Research/hu_hu_hu/sprint19/compspace/CompositionSpace


In [4]:
run = False
if run is True:
    data = ProcessPreparation(f"{MY_PROCESSED_DATA_PATH}/experiment_params.yaml")
    df_lst, files, ions, rrngs= data.get_apt_dataframe()
    # print(f"{df_lst}, type {type(df_lst)}")
    # print(files)
    # print(ions)
    # print(rrngs)

    for idx, file in enumerate(files):
        org_file = df_lst[idx]
        atoms_spec = []
        c = np.unique(rrngs.comp.values)
        for i in range(len(c)):
            range_element = rrngs[rrngs['comp']=='{}'.format(c[i])]
            total, count = data.atom_filter(org_file, range_element)
            name = i
            total["spec"] = [name for j in range(len(total))]
            atoms_spec.append(total)
    # print(atoms_spec)
    df_atom_spec = pd.concat(atoms_spec)
    # print(df_atom_spec)

<div class="alert alert-block alert-danger">Remove code in this cell obsolete thanks to paraprobe NeXus</div>

## Load positions and iontypes from paraprobe-transcoder and paraprobe-ranger results

In [5]:
RECONSTRUCTION_AND_RANGING = ("R21_08680-v02.pos", "R21_08680.rrng")
RECONSTRUCTION_AND_RANGING = ("PARAPROBE.Transcoder.Results.SimID.636502001.nxs",
                              "PARAPROBE.Ranger.Results.SimID.636502001.nxs")

In [6]:
# H5Web(RECONSTRUCTION_AND_RANGING[0])
# H5Web(RECONSTRUCTION_AND_RANGING[1])

In [8]:
with h5py.File(RECONSTRUCTION_AND_RANGING[0], "r") as h5r:
    trg = "/entry1/atom_probe/reconstruction/reconstructed_positions"
    xyz = h5r[trg][:, :]
    print(f"shape {np.shape(xyz)}, type {type(xyz)}, dtype {xyz.dtype}")

    trg = "/entry1/atom_probe/ranging/peak_identification"
    n_ion_types = len(h5r[trg])
    iontypes = {}
    for ion_id in np.arange(0, n_ion_types):
        iontypes[f"ion{ion_id}"] = (h5r[f"{trg}/ion{ion_id}/name"][()].decode("utf8"), np.uint8(ion_id))
    print(f"{n_ion_types}")
    for key, val in iontypes.items():
        print(f"{key}, {val}")

with h5py.File(RECONSTRUCTION_AND_RANGING[1], "r") as h5r:
    trg = "/entry1/iontypes/iontypes"
    ityp = h5r[trg][:]
    print(f"shape {np.shape(ityp)}, type {type(ityp)}, dtype {ityp.dtype}")

shape (4868202, 3), type <class 'numpy.ndarray'>, dtype float32
73
ion0, ('unknown iontype', 0)
ion1, ('C ++', 1)
ion2, ('C +', 2)
ion3, ('O +', 3)
ion4, ('Ti ++', 4)
ion5, ('Ti ++', 5)
ion6, ('Ti ++', 6)
ion7, ('Ti ++', 7)
ion8, ('Fe ++', 8)
ion9, ('Fe +', 9)
ion10, ('Fe ++', 10)
ion11, ('Fe ++', 11)
ion12, ('Fe ++', 12)
ion13, ('Fe +', 13)
ion14, ('Al ++', 14)
ion15, ('Al +++', 15)
ion16, ('Si ++', 16)
ion17, ('Si ++', 17)
ion18, ('Si ++', 18)
ion19, ('Ti O ++', 19)
ion20, ('Ti O ++', 20)
ion21, ('Ti O ++', 21)
ion22, ('Ti O ++', 22)
ion23, ('Ti O ++', 23)
ion24, ('Ti O +', 24)
ion25, ('Ti O +', 25)
ion26, ('Ti O +', 26)
ion27, ('Ti O +', 27)
ion28, ('Cr ++', 28)
ion29, ('Cr ++', 29)
ion30, ('Cr ++', 30)
ion31, ('Cr +', 31)
ion32, ('Cr +', 32)
ion33, ('Mn ++', 33)
ion34, ('Mn +', 34)
ion35, ('Co ++', 35)
ion36, ('Y +++', 36)
ion37, ('Y ++', 37)
ion38, ('Ga ++', 38)
ion39, ('Ga +', 39)
ion40, ('Cr O ++', 40)
ion41, ('Cr O ++', 41)
ion42, ('Cr O +', 42)
ion43, ('Y O ++', 43)
ion44, ('F

## Generate NXapm_composition_space results file

In [10]:
output_file_name = f"{MY_PROCESSED_DATA_PATH}/apm.composition.space.nxs"
with h5py.File(output_file_name, "w") as h5w:
    h5w.attrs["@NX_class"] = "NXroot"
    h5w.attrs["@file_name"] = f"{MY_PROCESSED_DATA_PATH}/{output_file_name}"
    h5w.attrs["@file_time"] = dt.datetime.now(dt.timezone.utc).isoformat()  # .replace("+00:00", "Z")
    # /@file_update_time
    h5w.attrs["@NeXus_repository"] = f"https://github.com/FAIRmat-NFDI/nexus_definitions/blob/get_nexus_version_hash()"
    h5w.attrs["@NeXus_version"] = f"get_nexus_version()"
    h5w.attrs["@HDF5_version"] = ".".join(map(str, h5py.h5.get_libversion()))
    h5w.attrs["@h5py_version"] = h5py.__version__
    
    trg = "/entry1"
    grp = h5w.create_group(trg)
    grp.attrs["NX_class"] = "NXentry"
    dst = h5w.create_dataset(f"{trg}/definition", data="NXapm_compositionspace_results")

In [11]:
H5Web(output_file_name)

<jupyterlab_h5web.widget.H5Web object>

## Import parameter via yaml

In [15]:
config_file_name = f"{MY_PROCESSED_DATA_PATH}/experiment_params.yaml"
with open(config_file_name, "r", encoding="utf-8") as yml:
    config = yaml.safe_load(yml)
    print(config)
    print(config["voxel_size"])

{'input_path': 'tests/data', 'output_path': 'tests/output', 'n_big_slices': 10, 'voxel_size': 2, 'bics_clusters': 10, 'n_phases': 2, 'ml_models': {'name': 'GaussianMixture', 'GaussianMixture': {'n_components': 2, 'max_iter': 100000, 'verbose': 0}, 'RandomForest': {'max_depth': 0, 'n_estimators': 0}, 'DBScan': {'eps': 3, 'min_samples': 5}}}
2


## Voxelize with rectangular transfer function without creating slices

In [16]:
# print(type(df_lst))
column_names = ['x', 'y', 'z']
# initialize extent (number of cells) along x, y, z axes
extent = [0, 0, 0]
# initialize min, max bounds for x, y, z
aabb3d = np.reshape([np.finfo(np.float32).max, np.finfo(np.float32).min,
          np.finfo(np.float32).max, np.finfo(np.float32).min,
          np.finfo(np.float32).max, np.finfo(np.float32).min], (3, 2), order="C")
# print(aabb3d)
n_ions = np.shape(xyz)[0]
voxel_identifier = np.asarray(np.zeros(n_ions), np.uint32)
print(f"shape {np.shape(voxel_identifier)}")
# edge length of cubic cells/voxels in nm
dedge = config["voxel_size"]
for axis_id in [0, 1, 2]:
    column_name = column_names[axis_id]
    # i = np.asarray(df_lst[0].loc[:, column_name], np.float32)
    aabb3d[axis_id, 0] = floor_to_multiple(np.min((aabb3d[axis_id, 0], np.min(xyz[:, axis_id]))), dedge)
    aabb3d[axis_id, 1] = ceil_to_multiple(np.max((aabb3d[axis_id, 1], np.max(xyz[:, axis_id]))), dedge)
    extent[axis_id] = np.uint32((aabb3d[axis_id, 1] - aabb3d[axis_id, 0]) / dedge)
    print(f"aabb3d {aabb3d[axis_id, :]}, extent {extent[axis_id]}")
    bins = np.linspace(aabb3d[axis_id, 0] + dedge, aabb3d[axis_id, 0] + (extent[axis_id] * dedge), num=extent[axis_id], endpoint=True)
    print(bins)
    if axis_id == 0:
        voxel_identifier = voxel_identifier + (np.asarray(np.digitize(xyz[:, axis_id], bins, right=True), np.uint32) * 1)
    elif axis_id == 1:
        voxel_identifier = voxel_identifier + (np.asarray(np.digitize(xyz[:, axis_id], bins, right=True), np.uint32) * np.uint32(extent[0]))
    else:
        voxel_identifier = voxel_identifier + (np.asarray(np.digitize(xyz[:, axis_id], bins, right=True), np.uint32) * np.uint32(extent[0]) * np.uint32(extent[1]))
print(voxel_identifier[0:10])
print(np.max(voxel_identifier))

shape (4868202,)
aabb3d [-32.  32.], extent 32
[-30. -28. -26. -24. -22. -20. -18. -16. -14. -12. -10.  -8.  -6.  -4.
  -2.   0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.
  26.  28.  30.  32.]
aabb3d [-32.  32.], extent 32
[-30. -28. -26. -24. -22. -20. -18. -16. -14. -12. -10.  -8.  -6.  -4.
  -2.   0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.
  26.  28.  30.  32.]
aabb3d [ 0. 72.], extent 36
[ 2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34. 36.
 38. 40. 42. 44. 46. 48. 50. 52. 54. 56. 58. 60. 62. 64. 66. 68. 70. 72.]
[3213 3397 1717 1448  555  398 3303 1608 1780 3707]
36847


In [21]:
## Create a sorted lookup table for all ion positions (implicitly specified by their evaporation_id) O(n*log(n)) time-complexity operation wi

In [22]:
ion_struct = [('iontype', np.uint8), ('voxel_id', np.uint32), ('evap_id', np.uint32)]
lu_ityp_voxel_id_evap_id = np.zeros(n_ions, dtype=ion_struct)
lu_ityp_voxel_id_evap_id["iontype"] = ityp[:]
del ityp
lu_ityp_voxel_id_evap_id["voxel_id"] = voxel_identifier
# del voxel_identifier
lu_ityp_voxel_id_evap_id["evap_id"] = np.asarray(np.linspace(1, n_ions, num=n_ions, endpoint=True), np.uint32)
print(lu_ityp_voxel_id_evap_id[0:10])
lu_ityp_voxel_id_evap_id = np.sort(lu_ityp_voxel_id_evap_id, kind="stable", order=["iontype", "voxel_id", "evap_id"])
print(lu_ityp_voxel_id_evap_id[0:10])
print(lu_ityp_voxel_id_evap_id[-10::])

[( 8, 3213,  1) ( 8, 3397,  2) ( 8, 1717,  3) ( 8, 1448,  4)
 ( 8,  555,  5) ( 8,  398,  6) (29, 3303,  7) ( 8, 1608,  8)
 ( 8, 1780,  9) (29, 3707, 10)]
[(0, 302,  2486) (0, 304,  3226) (0, 304,  5573) (0, 304, 13215)
 (0, 304, 17990) (0, 304, 18548) (0, 332,   173) (0, 333, 25249)
 (0, 334, 13610) (0, 334, 17234)]
[(72, 30824, 4001574) (72, 32290, 4331875) (72, 32443, 4375588)
 (72, 32540, 4116453) (72, 32911, 4739678) (72, 33091, 4520185)
 (72, 33866, 4518381) (72, 34491, 4730446) (72, 34555, 4758833)
 (72, 34967, 4856367)]


Report results of voxelization and metadata for the grid

In [18]:
# voxelization
with h5py.File(output_file_name, "a") as h5w:
    trg = "/entry1/voxelization"
    grp = h5w.create_group(trg)
    grp.attrs["NX_class"] = "NXprocess"
    dst = h5w.create_dataset(f"{trg}/sequence_index", data=np.uint64(1))

    trg = "/entry1/voxelization/cg_grid"
    grp = h5w.create_group(trg)
    grp.attrs["NX_class"] = "NXcg_grid"
    dst = h5w.create_dataset(f"{trg}/dimensionality", data=np.uint64(3))
    c = np.prod(extent)
    dst = h5w.create_dataset(f"{trg}/cardinality", data=np.uint64(c))
    dst = h5w.create_dataset(f"{trg}/origin", data=np.asarray([aabb3d[0, 0], aabb3d[1, 0], aabb3d[2, 0]], np.float64))
    dst.attrs["units"] = "nm"
    dst = h5w.create_dataset(f"{trg}/symmetry", data="cubic")
    dst = h5w.create_dataset(f"{trg}/cell_dimensions", data=np.asarray([dedge, dedge, dedge], np.float64))
    dst.attrs["units"] = "nm"
    dst = h5w.create_dataset(f"{trg}/extent", data=np.asarray(extent, np.uint32))  # max. 2*32 cells
    identifier_offset = 0
    dst = h5w.create_dataset(f"{trg}/identifier_offset", data=np.uint64(identifier_offset))  # start counting cells from 0

    voxel_id = identifier_offset
    position = np.zeros([c, 3], np.float64)
    for k in np.arange(0, extent[2]):
        z = aabb3d[2, 0] + (0.5 + k) * dedge
        for j in np.arange(0, extent[1]):
            y = aabb3d[1, 0] + (0.5 + j) * dedge
            for i in np.arange(0, extent[0]):
                x = aabb3d[0, 0] + (0.5 + i) * dedge
                position[voxel_id, :] = [x, y, z]
                voxel_id += 1
    dst = h5w.create_dataset(f"{trg}/position", compression="gzip", compression_opts=1, data=position)
    dst.attrs["units"] = "nm"
    del position

    voxel_id = identifier_offset
    coordinate = np.zeros([c, 3], np.uint32)
    for k in np.arange(0, extent[2]):
        for j in np.arange(0, extent[1]):
            for i in np.arange(0, extent[0]):
                coordinate[voxel_id, :] = [i, j, k]
                voxel_id += 1
    dst = h5w.create_dataset(f"{trg}/coordinate", compression="gzip", compression_opts=1, data=coordinate)
    del coordinate

In [20]:
# H5Web(output_file_name)

In [ ]:
## From this lookup we can very easily now compute the composition table

In [23]:
with h5py.File(output_file_name, "a") as h5w:
    trg = "/entry1/voxelization/cg_grid"
    dst = h5w.create_dataset(f"{trg}/voxel_identifier", compression="gzip", compression_opts=1, data=voxel_identifier)

    c = np.prod(extent)
    print(f"cardinality {c}")
    # now just add weight/counts for a the iontype-specific part of the lookup-table
    print(f"means we have to visit so that many entries in the lookup table {np.sum(lu_ityp_voxel_id_evap_id['iontype'] == 0)}")
    print(f"but by virtue of construction of the lookup table all the indices will be close in cache")
    total_weights = np.zeros(c, np.float64)
    for ityp in np.arange(0, n_ion_types):
        inds = np.argwhere(lu_ityp_voxel_id_evap_id["iontype"] == ityp)
        offsets = (np.min(inds), np.max(inds))
        # print(f"offsets {offsets}")
        # these are inclusive [min, max] array indices to use on lu_ityp_voxel_id_evap_id !
        
    # alternatively one could make two loops where in the first an offset lookup table is generated
    # after this point one can drop the iontype and evap_id columns from the lu_ityp_voxel_id_evap_id lookup table
        ityp_weights = np.zeros(c, np.float64)
        for offset in np.arange(offsets[0], offsets[1] + 1):
            idx = lu_ityp_voxel_id_evap_id["voxel_id"][offset]
            ityp_weights[idx] += 1.
        # print(f"ityp {ityp}, np.sum(ityp_weights) {np.sum(ityp_weights)}")
        
        # atom/molecular ion-type-specific contribution/intensity/count in each voxel/cell
        trg = f"/entry1/voxelization/ion{ityp}"
        grp = h5w.create_group(f"{trg}")
        grp.attrs["NX_class"] = "NXion"
        dst = h5w.create_dataset(f"{trg}/name", data=iontypes[f"ion{ityp}"][0])
        dst = h5w.create_dataset(f"{trg}/weight", compression="gzip", compression_opts=1, data=ityp_weights)
        dst.attrs["units"] = "a.u."  
        
        total_weights += ityp_weights
        print(f"ityp {ityp}, np.sum(total_weights) {np.sum(total_weights)}")
    print(f"cardinality of cg_grid {c}, n_ions {n_ions}")

    # total atom/molecular ion contribution/intensity/count in each voxel/cell
    trg = f"/entry1/voxelization"
    dst = h5w.create_dataset(f"{trg}/total", compression="gzip", compression_opts=1, data=total_weights)
    dst.attrs["units"] = "a.u."

# reload weights and compute to compositions if really needed as compositions
# dst = h5w.create_dataset(f"{trg}/composition", compression="gzip", compression_opts=1, data=###)
# dst.attrs["unit"] = "a.u."

cardinality 36864
means we have to visit so that many entries in the lookup table 238176
but by virtue of construction of the lookup table all the indices will be close in cache
ityp 0, np.sum(total_weights) 238176.0
ityp 1, np.sum(total_weights) 238286.0
ityp 2, np.sum(total_weights) 238571.0
ityp 3, np.sum(total_weights) 241498.0
ityp 4, np.sum(total_weights) 243540.0
ityp 5, np.sum(total_weights) 245455.0
ityp 6, np.sum(total_weights) 264818.0
ityp 7, np.sum(total_weights) 266245.0
ityp 8, np.sum(total_weights) 3816576.0
ityp 9, np.sum(total_weights) 3818363.0
ityp 10, np.sum(total_weights) 4060876.0
ityp 11, np.sum(total_weights) 4143065.0
ityp 12, np.sum(total_weights) 4157279.0
ityp 13, np.sum(total_weights) 4157480.0
ityp 14, np.sum(total_weights) 4159051.0
ityp 15, np.sum(total_weights) 4159215.0
ityp 16, np.sum(total_weights) 4162639.0
ityp 17, np.sum(total_weights) 4162859.0
ityp 18, np.sum(total_weights) 4163017.0
ityp 19, np.sum(total_weights) 4164222.0
ityp 20, np.sum(tota

In [ ]:
# For a large number of voxels, say a few million and dozens of iontypes storing all
# ityp_weights in main memory might not be useful, instead these should be stored in the HDF5 file
# inside the loop and ones the loop is completed, i.e. each total weight for each voxel known
# we should update the data in the HDF5 file, alternatively one could also just store the
# weights instead of the compositions and then compute the composition with a linear in c*ityp time
# complex division, there are even more optimizations one could do, but probably using
# multithreading would be a good start before dwelling deeper already this code here is
# faster than the original one despite the fact that it works on the entire portland wang
# dataset with 4.868 mio ions, while the original test dataset includes only 1.75 mio ions
# the top part of the dataset also the code is much shorter to read and eventually even
# more robust wrt to how ions are binned with the rectangular transfer function
# one should say that this particular implementation (like the original) one needs
# substantial modification when one considers a delocalization kernel which spreads
# the weight of an ion into the neighboring voxels, this is what paraprobe-nanochem does
# one can easily imagine though that the results of this voxelization step can both be
# fed into the composition clustering step and here is then also the clear connection
# where the capabilities for e.g. the APAV open-source Python library end and Alaukik's
# ML/AI work really shines, in fact until now all code including the slicing could have
# equally been achieved with paraprobe-nanochem.
# Also the excessive reimplementation of file format I/O functions in datautils should
# be removed. There is an own Python library for just doing that more robustly and
# capable of handling all sorts of molecular ions and charge state analyses included

In [24]:
H5Web(output_file_name)

<jupyterlab_h5web.widget.H5Web object>